# Path Variables

In [ ]:
import pandas as pd
import numpy as np
import nltk
#from nltk.tokenize import word_tokenize
from nltk import sent_tokenize, word_tokenize
from enchant.checker import SpellChecker

# https://www.packtpub.com/mapt/book/application-development/9781782167853/2/ch02lvl1sec23/spelling-correction-with-enchant
import enchant
from nltk.metrics import edit_distance
from nltk.tokenize.stanford import StanfordTokenizer
import string
import time
import os

nltk_data_dir = "/home/elkhand/datasets/nltk_data"
if not os.listdir(nltk_data_dir):
    nltk.download(download_dir=nltk_data_dir,quiet=True) # Download once
nltk.data.path.append(nltk_data_dir)

rootPathOfGlove = "/home/elkhand/datasets/glove-vectors/"

gloveVectorPath = '/home/elkhand/datasets/glove-vectors/glove.6B.300d.txt'

#glvocab_300Path = "/home/elkhand/datasets/glove-vectors/glvocab_300.txt"
#glwordvectors_1_300Path = "/home/elkhand/datasets/glove-vectors/glwordvectors_1_300.txt"

quoraDataSetPath = '/home/elkhand/datasets/Quora/data/quora_duplicate_questions.tsv'

derivedDataSetPathRoot = "/home/elkhand/datasets/Quora/derived/"

EOS_CHAR = 'zzzz'

# Creating train/dev/test set
## conll format

In [ ]:
print 'loading data...'
quora = pd.read_csv(quoraDataSetPath, sep='\t', encoding='utf-8')
print quora.shape
print quora.head(2)

# replacer = SpellingReplacer()
np.random.seed(0)

def tokenize_quora_row(row, file_dict):
    # tokenize
    try:
        q1_tok = word_tokenize(row['question1'])
        q2_tok = word_tokenize(row['question2'])
        conll_entry_a = '\n'.join(q1_tok) + '\n\n'
        conll_entry_a = conll_entry_a.encode('utf-8')
        conll_entry_b = '\n'.join(q2_tok) + '\n\n'
        conll_entry_b = conll_entry_b.encode('utf-8')        
        label_entry = str(row['is_duplicate'])+'\n'
    except Exception, e:# encoding errors, not str, etc.
        print repr(e)  
        return
    
    # split train/dev/test
    x = np.random.random()
    if x < 0.98: # .98
        k = 'train'
    elif x < 0.99: # .99
        k = 'dev'
    else: #0.01
        k = 'test'
    
    # write
    fa, fb, flab = file_dict[k]['fa'], file_dict[k]['fb'], file_dict[k]['flab']
    fa.write(conll_entry_a)
    fb.write(conll_entry_b)
    flab.write(label_entry)

    
# tokenize conll
with open(derivedDataSetPathRoot + 'data/dat_train_a.conll', 'w') as trainaf:
    with open(derivedDataSetPathRoot + 'data/dat_train_b.conll', 'w') as trainbf:
        with open(derivedDataSetPathRoot + 'data/labels_train.conll', 'w') as trainlabelf:
            with open(derivedDataSetPathRoot + 'data/dat_dev_a.conll', 'w') as devaf:
                with open(derivedDataSetPathRoot + 'data/dat_dev_b.conll', 'w') as devbf:
                    with open(derivedDataSetPathRoot + 'data/labels_dev.conll', 'w') as devlabelf:        
                        with open(derivedDataSetPathRoot + 'data/dat_test_a.conll', 'w') as testaf:
                            with open(derivedDataSetPathRoot + 'data/dat_test_b.conll', 'w') as testbf:
                                with open(derivedDataSetPathRoot + 'data/labels_test.conll', 'w') as testlabelf:
                                    file_dict = {
                                        'train': {'fa': trainaf, 'fb': trainbf, 'flab': trainlabelf},
                                        'dev': {'fa': devaf, 'fb': devbf, 'flab': devlabelf},
                                        'test': {'fa': testaf, 'fb': testbf, 'flab': testlabelf}
                                    }
                                    tok_conll = quora.apply(lambda x: tokenize_quora_row(x, file_dict), axis=1)

# MungeGlove

Generates glvocab_1.txt and glwordvectors_1_[100/300].txt from the GloVe data.

In [ ]:
# Generates glvocab_1.txt and glwordvectors_1_[100/300].txt from the GloVe data.
#
# Download the GloVe data from http://nlp.stanford.edu/data/glove.6B.zip

vocab = None

for embed_size in [100, 300]:
    current_vocab = []

    with open(rootPathOfGlove + "glove.6B.%dd.txt" % embed_size, "r") as f_in:
        with open(rootPathOfGlove + "glwordvectors_%d.txt" % embed_size, "w") as f_out:
            for line in f_in:
                cols = line.split()
                assert len(cols) == embed_size + 1

                word = cols[0]
                current_vocab.append(word)
                f_out.write(line[len(word)+1:])

    if vocab is None:
        vocab = current_vocab
    else:
        assert vocab == current_vocab

assert len(vocab) == 400000

with open(rootPathOfGlove + "glwordvectors_common.txt", "w") as f:
    for word in vocab:
        f.write(word + "\n")

# Spell checking
** NOTE: ** Not used

In [ ]:


class SpellingReplacer(object):
    def __init__(self, dict_name='en', max_dist=2):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist

    def replace(self, words):
        corrected_words = list()
        for word in words:
            if self.spell_dict.check(word):
                corrected_words.append(word)
                continue
            suggestions = self.spell_dict.suggest(word)
            if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
                corrected_words.append(suggestions[0])
            else:
                corrected_words.append(word)
    return corrected_words


class TokenizerWorker(object):
    
    def __init__(self, gl_vocab_index, gl, spellreplacer):
#         self.tokenizer = tokenizer
        self.gl_vocab_index = gl_vocab_index
        self.gl = gl
        self.spellreplacer = spellreplacer
    
    def tokenize(self, s):
#         tokens = self.tokenizer.tokenize(s)
#         tokens =  re.findall(r"[\w']+", s)
        try:
            s = re.sub(r'[^\w\s]','', s.encode('utf-8').lower())
        except:
            return [s]
        tokens = word_tokenize(s) # lower and strip punctuation
        tokens_postprocess = tokens
        return tokens_postprocess
    
    def __call__(self, s):

        tokens = self.tokenize(s)
        s_vect = np.zeros((1, self.gl.shape[1]))
        missing_word = False
        
        for word in tokens: # sum glove vectors
            try:
                s_vect += self.gl.iloc[self.gl_vocab_index.ix[word], :]
            except KeyError:
                missing_word = True
            except TypeError:
                continue
        
        if missing_word: # rerun with spellcheck
            tokens = self.spellreplacer.replace(tokens)
            s_vect = np.zeros((1, self.gl.shape[1]))
            for word in tokens:
                try:
                    s_vect += self.gl.iloc[self.gl_vocab_index.ix[word], :]
                except (KeyError, TypeError) as e:
                    pass
            s = ' '.join(tokens)

        return (s, s_vect)
    

replacer = SpellingReplacer()

# load glove pretrained vectors
gl = pd.read_csv(gloveVectorPath, header=None, sep='\s+', error_bad_lines=False, quoting=csv.QUOTE_NONE, encoding='utf8')
gl_vocab = gl.pop(0)
gl_vocab_index = pd.DataFrame({'vocab_idx': gl_vocab.index})
gl_vocab_index.index = gl_vocab.values
print gl_vocab.shape, gl.shape
gl_vocab_index.head(5)

